In [1]:

import torch
import yaml

from prithvi_100m.Prithvi import MaskedAutoencoderViT
from dataset import XBDDataset

/Users/michaelskaug/venv/prithvi-change-detection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Model

In [2]:
# load weights
weights_path = "./prithvi_100m/Prithvi_100M.pt"
checkpoint = torch.load(weights_path, map_location="cpu")

# read model config
model_cfg_path = "./prithvi_100m/Prithvi_100M_config.yaml"
with open(model_cfg_path) as f:
    model_config = yaml.safe_load(f)

model_args = model_config["model_args"]
train_args = model_config["train_params"]

# 2 frames: pre image and post image
model_args["num_frames"] = 2

# instantiate model
model = MaskedAutoencoderViT(**model_args)
model.eval()

# load weights into model
del checkpoint['pos_embed']
del checkpoint['decoder_pos_embed']
_ = model.load_state_dict(checkpoint, strict=False)

# Data

In [3]:
dataset = XBDDataset('data/geotiffs/tier1')